<a href="https://colab.research.google.com/github/amitchug/ALMlops/blob/main/M2_AST_02_CNN_Transfer_Learning_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification Programme in AI and MLOps
## A programme by IISc and TalentSprint
### Assignment 2: Leveraging a pre-trained model

## Learning Objectives:

At the end of the experiment, you will be able to:

1. Understand and use a pre-trained model
2. Fine-tune the top layers while using a pre-trained model


## Introduction

A common and highly effective approach to deep learning on small image datasets is to use a pre-trained model.

If the original dataset is large enough and general enough, the spatial hierarchy of features learned by the pre-trained model can effectively act as a generic model of the visual world, and hence, its features can prove useful for many different computer vision problems even though these new problems may involve completely different classes than those of the original task.

There are two ways to use a pre-trained model:
  * feature extraction and
  * fine-tuning

### Feature extraction

A CNN typically consists of a:
* Convolutional base
* Densely connected classifier

Key Idea -
* Features are learned by the convolutional base. So reuse it.
* Train a new classifier for your problem

![picture](https://drive.google.com/uc?export=view&id=12FzJVGUCzQGDArYSqdABntKwKGHzIjVX)

### Setup Steps:

In [ ]:
#@title Please enter your registration id to start: { run: "auto", display-mode: "form" }
Id = "" #@param {type:"string"}

In [ ]:
#@title Please enter your password (your registered phone number) to continue: { run: "auto", display-mode: "form" }
password = "" #@param {type:"string"}

In [ ]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython

ipython = get_ipython()

notebook= "M2_AST_02_CNN_Transfer_Learning_A" #name of the notebook

def setup():
#  ipython.magic("sx pip3 install torch")

    # ipython.magic("wget https://cdn.iisc.talentsprint.com/AIandMLOps/Datasets/Acoustic_Extinguisher_Fire_Dataset.xlsx")
    from IPython.display import HTML, display
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    ipython.magic("notebook -e "+ notebook + ".ipynb")

    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:
        print(r["err"])
        return None
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getAnswer() and getComplexity() and getAdditional() and getConcepts() and getComments() and getMentorSupport():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional,
              "concepts" : Concepts, "record_id" : submission_id,
              "answer" : Answer, "id" : Id, "file_hash" : file_hash,
              "notebook" : notebook,
              "feedback_experiments_input" : Comments,
              "feedback_mentor_support": Mentor_support}
      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:
        print(r["err"])
        return None
      else:
        print("Your submission is successful.")
        print("Ref Id:", submission_id)
        print("Date of submission: ", r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://aimlops-iisc.talentsprint.com/notebook_submissions")
        #print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
        return submission_id
    else: submission_id


def getAdditional():
  try:
    if not Additional:
      raise NameError
    else:
      return Additional
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None

def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None


# def getWalkthrough():
#   try:
#     if not Walkthrough:
#       raise NameError
#     else:
#       return Walkthrough
#   except NameError:
#     print ("Please answer Walkthrough Question")
#     return None

def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None


def getMentorSupport():
  try:
    if not Mentor_support:
      raise NameError
    else:
      return Mentor_support
  except NameError:
    print ("Please answer Mentor support Question")
    return None

def getAnswer():
  try:
    if not Answer:
      raise NameError
    else:
      return Answer
  except NameError:
    print ("Please answer Question")
    return None


def getId():
  try:
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
else:
  print ("Please complete Id and Password cells before running setup")



### Import libraries

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D,MaxPool2D
from tensorflow.keras import Input
from tensorflow.keras import Model
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Let us reuse the VGG16 network which has been trained on the ImageNet, which contains multiple classes of cats and dogs among other things.

We expect the convolution base to have learned features that help it identify cats and dogs.

Keras provides us with a pre-trained VGG16 network!



In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
# Load the convolutional base
conv_base = VGG16(
    weights = "imagenet",
    include_top = False, # Don't re-use the classifier (top layers)
    input_shape = (180,180,3)
)

In [ ]:
# See a summary of the convolutional base
# YOUR CODE HERE

So how do we extract the features? Simple:

Pass the images through the convolutional base.

In [ ]:
#@title Download the data
!wget -qq https://cdn.iisc.talentsprint.com/AIandMLOps/Datasets/cats_vs_dogs_small.zip
!unzip -qq '/content/cats_vs_dogs_small.zip'

In [ ]:
# defining path names for futur use
data_dir = '/content/cats_vs_dogs_small'

train_path = data_dir + '/train'
validation_path = data_dir + '/validation'
test_path = data_dir + '/test'

### Converting the image dataset into a workable format

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory
# YOUR CODE HERE

### Passing the dataset through Conv Base i.e forward pass through pre-trained weights

Before forward pass, Preprocessing the dataset specific to VGG16 is also required.

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input # Importing function for preprocessing specific to the vgg16

In [ ]:
# Extracting the features from pretrained models
def get_features_and_labels(dataset):
            # YOUR CODE HERE                                 # preprocessing specific to the vgg16
            # YOUR CODE HERE                                 # forward pass
            # YOUR CODE HERE

# Doing the same for all datasets
train_feature, train_labels = get_features_and_labels(train_dataset)
val_feature, val_labels = get_features_and_labels(validation_dataset)
test_feature, test_labels = get_features_and_labels(test_dataset)

### Defining and training the densely connected classifier



In [ ]:
# Q: Defining the classifier using the Functional API
# YOUR CODE HERE
model_without_conv_base = Model(inputs,outputs)  ## Model is imported from keras while importing libraries

# YOUR CODE HERE

#### Call Back Function

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# Define a function to return a commmonly used callback_list
# YOUR CODE HERE

In [ ]:
# fit the densely connected classsifier
# YOUR CODE HERE

Plotting Accuracy and Loss vs Epochs

In [ ]:
import pandas as pd
# YOUR CODE HERE

In [ ]:
test_model = load_model("feature_extraction_keras")
test_loss, test_acc = test_model.evaluate(test_feature,test_labels)
print(f"Test accuracy:{test_acc:.3f}")

The pretrained conv base + newly trained classification head has achieved roughly **97%** accuracy!

Remember, for convnet_from_scratch_with_data_augmentation, it was ~ 85%

For convnet_from_scratch, it was ~ 70%

## Fine tuning

The key idea here is to fine tune some top layers of the conv base as well.

We do so by freezing most of the bottom layers, leaving only a few top layers to train. Lets see:

In [ ]:
# Load the convolutional base
conv_base = VGG16(
    weights = "imagenet",
    include_top = False, # Don't re-use the classifier (top layers)
    input_shape = (180,180,3)
)

In [ ]:
#Define the model
inputs = Input(shape=(180, 180, 3))
# x = data_augmentation(inputs)
x = preprocess_input(inputs)
x = conv_base(x)
x = Flatten()(x)
x = Dense(256)(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation="sigmoid")(x)
model = Model(inputs, outputs)

In [ ]:
from tensorflow.keras.optimizers import RMSprop

In [ ]:
#Freezing all layers until the fourth from the last
conv_base.trainable = True
for layer in conv_base.layers[:-4]:   #[1,2,3,4,5,6,7,8,9]
    layer.trainable = False

#  Model compilation and summary
# YOUR CODE HERE

In [ ]:
## Callback function
callbacks = [ModelCheckpoint(filepath="fine_tuning_keras", save_best_only=True, monitor="val_loss")]

In [ ]:
## Training
history = model.fit(train_dataset, epochs=30, validation_data=validation_dataset, callbacks=callbacks)

In [ ]:
## Evaluation
model = load_model("fine_tuning_keras")
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

# OPTIONAL EXERCISE: plot the losses by using appropriate callbacks and tensorboard

# Using already run and saved models

For this save the model checkpoints in your drive (download the checkpoints and save to drive) give that path while loading.

Mount your G drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/"

For example, you downloaded the `feature_extraction_keras` model and saved it in your drive inside a folder named 'model'. You can load it here like this:

    model_saved_1 = load_model('/content/drive/MyDrive/model/feature_extraction_keras')

 In this case the file path is simply : '/content/feature_extraction_keras'

In [ ]:
model_fewa = load_model('/content/feature_extraction_keras')

In [ ]:
test_loss, test_acc = model_fewa.evaluate(test_feature, test_labels)
print(f"Test accuracy is:{test_acc:.3f}")

Great! feature_extraction model has achieved **97%** accuracy!

### Please answer the questions below to complete the experiment:




In [ ]:
#@title  What is the main idea of transfer learning in deep learning? { run: "auto", form-width: "500px", display-mode: "form" }
Answer = "" #@param ["", "A) To train the pre-trained Convolutional Base (Conv_Base) from scratch on a new dataset or task.", "B) To use the pre-trained Convolutional Base (Conv_Base) as a feature extractor and add new layers to the top of the pre-trained model for fine-tuning on a new dataset or task.", "C) To use the pre-trained Convolutional Base (Conv_Base) as the only layers for the new dataset or task without any modifications."]

In [ ]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [ ]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "" #@param {type:"string"}


In [ ]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "" #@param ["","Yes", "No"]


In [ ]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Mentor Support: { run: "auto", vertical-output: true, display-mode: "form" }
Mentor_support = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id = return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")